# *This notebook is only for illustration and cannot be run.*

## `GalaxyGenius-MSA`

`GalaxyGenius-MSA` simulates observations of JWST MSA-3D using TNG hydrodynamical simulations and SKIRT radiative transfer project.

### Workflow
- Preprocessing: extract necessary particles, incorporating stars, star-forming regions and dust (if dust is included), from TNG hydrodynamical simulations and save them in particle files. Simultenously, the ski file for SKIRT exection is also generated.
- Generation: run SKIRT radiative transfer based on particles files and ski file, and generate data cubes in (wavelength, spatial, spatial) and SEDs.
- Postprocess: process the data cube to match the observational conditions of JWST MSA, including dispersion, transmission and resolution. The mock observation is a pseduo-IFU, with output also a data cube. 

### Imports

If you do not install `galaxyGeniusMSA` by `pip install .`, you should manually set the base path as:

In [1]:
import sys
sys.path.append('..') # change the path to where galaxyGeniusMSA exists

In [2]:
from galaxyGeniusMSA.config import Configuration
from galaxyGeniusMSA.preprocess import PreProcess
from galaxyGeniusMSA.generation import DataGeneration
from galaxyGeniusMSA.postprocess import PostProcess

# This is the Units singleton, which determines unit convention for hydro-simulation at different redshift
from galaxyGeniusMSA.utils import Units

In [ ]:
import os
import json
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
import astropy.units as u
from astropy.cosmology import Planck15
import h5py

### Set paths

`GALAXYGENIUS_DATA_DIR` sets the Data director for `galaxyGeniusMSA`, which stores config and SKIRT ski templates. In addition, files for transmission and dispersion are also stored.  
`STPSF_PATH` sets the necessary files to generate PSFs for various instrument of JWST.

If you do not set environment variables `GALAXYGENIUS_DATA_DIR` and `STPSF_PATH` in `.bashrc`, use following method to set them:

In [ ]:
os.environ['GALAXYGENIUS_DATA_DIR'] = 'your/path/to/galaxyGeniusMSA/Data'
os.environ['STPSF_PATH'] = 'your/path/to/stpsf-data'

### Manage configurations

configurations control the complete workflow of the simulation. They are divided into two parts, main configuration in `Data/config/config.toml` and mock observation related in `Data/config/config_MSA.toml`. Please check the two files for more details.

In [ ]:
# initialize configuration class
config = Configuration()

# get configuration
conf = config.get_config()

`conf` is a dict, some values are with astropy units, therefore, do not forget the unit when resetting some values.

In [ ]:
conf['includeDust'] = True
conf['simulationMode'] = 'ExtinctionOnly'
conf['includeVelocity'] = True
conf['snapNum'] = 50
conf['snapRedshift'] = 1.0
conf['viewRedshift'] = 1.0
conf['oversample'] = 4

conf['starformingSEDFamily'] = 'MAPPINGS'
conf['ageThreshold'] = 10 * u.Myr
conf['minWaveRT'] = 0.1 * u.um
conf['maxWaveRT'] = 2.0 * u.um
conf['numWaveRT'] = 3000

conf['minWaveOutput'] = 0.97 * u.um
conf['maxWaveOutput'] = 1.82 * u.um
conf['numWaveOutput'] = 3000

conf['faceAndEdge'] = False
conf['randomViews'] = False
conf['numViews'] = 1
conf['inclinations'] = [0]
conf['azimuths'] = [0]

After resetting some configurations, save them in current directory in `.toml` file:

In [ ]:
config.save_config(conf)

# reload from the toml file in current directory, not necessary
conf = config.get_config()

### Data Preprocessing

Preprocessing extract particles from hydrodynamical simulations and generate SKIRT ski file.

There are four approaches for extraction, corresponding to different data format:  
- From snapshots: this approach requires group catalog, snapshot and offset files stored in local. These files are read by `illustris_python`, therefore, the directory tree must follow the demand of `illustris_python`:
```Bash
    TNGPath  
    ├── postprocessing/  
    │   └── offsets/  
    │       └── offsets_{N}.hdf5  
    └── snapshots/  
        ├── groups_{N}/  
        │   └── fof_subhalo_tab_{N}.{n}.hdf5  
        └── Snapshot_{N}/  
            └── snap_{N}.{n}.hdf5  
```  
- From web-based API: this approach is activated by setting `config['requests']` to be `True` and a valid `config['apiKey']` is provided. It employs the web-based API of TNG to retrieve data, therefore, no file mentioned above is required. However, the request speed is slow and may fail sometimes.  
- From subhalo particle file: particle file and metadata of each subhalo in each snapshot can be individually downloaded using web-based API. The particle file is much smaller than the complete snapshot. This approach accepts subhalo particle file in `hdf5` format and metadata in `json` format. The subhalo downloading program is in `tutorial/download_subhalos.py`. 
- Input particles directly: this approach accepts extracted particles already loaded in memory. Note that the units of each property is required. This method is designed for hydrodynamical simulations except Illustris and Illustris-TNG, such as EAGLE.

Before initialize `PreProcess` class, the unit convention should be firstly defined, since in TNG simulations, the unit is in `ckpc/h`, `Msun/h` and so on. `c` and `h` represent comoving and hubble constant respectively, which can be different in distinct redshift and cosmological model, and all the calculations in `PreProcess` will follow the defined unit convention.  
If you do not define, don't worry, the unit convention will be initialized by Planck15 and the redshift provided by `config['snapRedshift']`.

In [ ]:
# Units is a singleton. It is the same for all calculations in `PreProcess`
units = Units(snapRedshift=conf['snapRedshift'], cosmology=Planck15)

In [ ]:
# initialize PreProcess class
preprocess = PreProcess(conf)

- The first approach (from snapshots stored in local)

The subhaloIDs and SFRs for subhalos following the mass range defined by `config['minStellarMass']` and `config['maxStellarMass']` are extracted from the snapshot.  
And then we specify one subhalo to obtain their metadata and automatically save them in a `workingDir/Subhalo_{subhaloID}.json` file.

In [ ]:
subhalos = preprocess.get_subhalos()
subhalo_info = preprocess.subhalo(subhalos['SubhaloID'][0])

- The second approach (from web-based API)

This approach is activated by setting `config['requests']` to be `True` and a valid `config['apiKey']` is provided.  
The subhaloIDs and SFRs for subhalos following the mass range defined by `config['minStellarMass']` and `config['maxStellarMass']` are extracted from the web-based API.  
And then we specify one subhalo to obtain their metadata and automatically save them in a `workingDir/Subhalo_{subhaloID}.json` file.

In [ ]:
subhalos = preprocess.get_subhalos()
subhalo_info = preprocess.subhalo(subhalos['SubhaloID'][0])

- The third approach (from subhalo particle file)

This approach should call `inputSubhaloParticleFile` method, which accepts three inputs: `subhaloParticleFile (str)`, `subhaloInfo (dict | None)` and `subhaloInfoFile (str | None)`.  

`subhaloParticleFile`: should be in `h5` format downloaded by `tutorial/download_subhalos.py`;  
`subhaloInfo`: The required parameters for `subhaloInfo` is `id` (subhaloID), `halfmassrad_stars` (radius at half stellar mass), `pos_x`, `pos_y` and `pos_z` (center position of the subhalo). Other parameters are optional, however it affects the calculation of true properties in `PostProcess`. Consequently, setting this as `None` and input by `subhaloInfoFile` is a better choice;  
`subhaloInfoFile` should be in `json` format, which is also downloaded by `tutorial/download_subhalos.py` along with particle file.

In [ ]:
subhaloID = 0
subhalo_file = os.path.join(
    f'../data/snapshots-{conf["snapNum"]}', 
    f'subhalo-{subhaloID}/subhalo_{subhaloID}_particles.h5')
subhalo_info_file = os.path.join(
    f'../data/snapshots-{conf["snapNum"]}', 
    f'subhalo-{subhaloID}/subhalo_{subhaloID}.json')

In [ ]:
preprocess.inputSubhaloParticleFile(subhalo_file, subhaloInfoFile=subhalo_info_file)

After specifying or loading the data for the considered subhalo, we should call `prepare()` to generate particle files including stars, star-forming regions and dust (if incorporated) and SKIRT ski file.  
`prepare()` can take in a dict of some arguments, which corresponds to the configurations, thus the configurations can be modified before generating necessary file for SKIRT. Note that, however, the `config.toml` and `config_MSA.toml` do not change when modifying configurations by this method. 

In [ ]:
arguments = {}
arguments['faceAndEdge'] = False
arguments['numViews'] = 1
arguments['randomViews'] = False
arguments['inclinations'] = [0]
arguments['azimuths'] = [0]

preprocess.prepare(arguments)

Once `prepare()` finishes, the following files will appear in the workingDir:  
`config.json`: updated configurations and information for subhalo  
`dust.txt`: dust particles, if included  
`starforming_regions.txt`: star forming regions  
`stars.txt`: star particles  
`Subhalo_{subhaloID}_particles.h5`: particle properties used in `PreProcess`
`Subhalo_{subhaloID}.json`: subhalo metadata

### Data cube generation

Data cube generation runs SKIRT based on the files mentioned above. The execution takes a well. The outputs are data cube (wavelength, spatial, spatial) and SED for each observing direction. After execution, necessary files are moved to a new directory `dataCubes`

In [ ]:
# initialize DataGeneration class
dataGeneration = DataGeneration(conf)

# run data generation
dataGeneration.runSKIRT()

### Postprocess

postprocess convert a specific data cube to mock MSA-3D observation. Essentially, this is an IFU simulation, which simulates spectra for each pixel based on the observing conditions of JWST MSA, and assemble them as an IFU data cube.

In [ ]:
# initialize PostProcess class
postprocess = PostProcess(subhaloID)

We define several manual input methods for `PostProcess` class:  

`input_bkg_interp(interp_bkg: interp1d)`: input background interpolation function (angstrom vs. MJy/sr). If not called, the interpolation will be obtained by JWST Background Tools (JBT) based on the `obsRA`, `obsDec`, `threshold` and `thisDay` defined in config.

`input_psf(psf_cube: np.ndarray)`: input PSF cube (`numpy.ndarray`), make sure that this cube matches the `pixelScale` and `oversamp` in config. If not called, the PSF cube will be obtained using STPSF considering the `disperser`, `filter` and `oversamp` defined in config and wavelength grids for data cube. The PSF modeling process can be slow.

`input_subhalo_info(subhalo_info: dict)`: input subhalo info (`dict`), to update the metadata of subhalo.

`input_particle_file(particle_file: str)`: input particle filename (`str`), instead of using the one saved by `PreProcess`.  

`input_dataCube(dataCube_path: str, viewing_angle: Union[list, None]=None)`, input datacube path (`str`) and the viewing angle (`list`). This method should be called after the previous input methods. If the viewing_angle is set as None, the viewing angle will be obtained from `config.json`.

In [ ]:
# input data cube, must be called
postprocess.input_dataCube(dataCube_path='your/datacube/path/generated/by/SKIRT', 
                          viewing_angle=None)

After input the data cube, we are ready to simulate the MSA data tensor. The data tensor will be in 4-dimension (spatial, spatial, 4, wavelength), where the 4 means wavelength, flux, flux error, and SED. Therefore, we technically call it as dataTensor. The dataTensor will be saved in `MSA_mock/Subhalo_{subhaloID}`.

In [ ]:
postprocess.create_MSA_dataTensor()

Next, we display the MSA mock, including the exposure array, mock observation configuration, several spectra. 

In [ ]:
postprocess.illustrate_MSA()

In order for calibration, we need several properties that can be obtained by fitting spectra and compare with the truth.  
We provide several default properties, including 'MetallicityStarFormingRegion', 'MetallicityGas', 'LOSVelocity', 'Mass', 'VelDisp' and 'NumberStarformingRegion'.  
You can also define your own properties, the following shows the procedure:  

1. Define the property function following the template

In [ ]:
from galaxyGeniusMSA.properties import (
    age_interp, 
    rotate_coordinates,
    transform,
    calc_stats
)

def property_xx(
    particle_file: str, inclination: np.float32, azimuth: np.float32, 
    bins_perpendicular: np.ndarray, bins_parallel: np.ndarray, 
    transformation: dict, subhalo_info: dict, configs: dict) -> tuple[np.ndarray, str]:
    
    """
    The template function for calculating specific property. 
    Note that the inner operation can be customized, but the parameters should be kept the same.
    The pipeline is as follows:
    1. Read the particle file and obtain the properties;
    2. Rotate the coordinates to the observing direction;
    3. Apply the transformation defined in config;
    4. Calculate the statistics for considered properties (e.g. count, mean, std, sum);
    5. Return the statistics and unit.
    
    - particle_file: 
        The particle file to be used for property. 
        If postprocess.input_particle_file() is not called, the particle file saved in preprocess will be used.  
    - inclination and azimuth:
        The inclination and azimuth used in SKIRT, used to rotate the coordinates of particles to the observing direction.
    - bins_perpendicular and bins_parallel:
        The bins for perpendicular and parallel directions to match the slit stepping and dither. 
        Note that the units are in kpc. 
    - transformation:
        A dict storing rotate angle, shift in perpendicular and parallel directions.
    - subhalo_info:
        A dict storing the subhalo information, including SFR, total stellar, DM mass, etc.
    - configs:
        A dict storing the settings from config.toml and config_MSA.toml.
    """
    
    # if need age information
    fage = age_interp(configs['cosmology']) 
    
    particles = {}
    
    # particle file is in h5 format, 
    # either use the one saved in preprocess or manually input.
    with h5py.File(particle_file, 'r') as file:
        
        # coordinates for stellar particles
        # conversion to np.float32 is necessary for numba function to work directly without recompliation
        particles['Coordinates'] = file['PartType4']['Coordinates'][:].astype(np.float32)
        
        # note that for coordinates, the unit should be kpc
        # if you use downloaded particle file from TNG, the default unit is ckpc/h 
        # units.position is ckpc/h, and need to convert to kpc
        particles['Coordinates'] = (particles['Coordinates'] * units.position).to_value(u.kpc)
        
        # if input_particle_file is not called, you use the particle file saved in preprocess,
        # the unit is already in kpc and saved as file['partType4']['Coordinates].attrs['unit']
        # so you can directly use particles['Coordinates'][:].astype(np.float32), or use
        # unit = file['partType4']['Coordinates'].attrs['unit']
        # particles['Coordinates] = (particles['Coordinates'] * u.Unit(unit)).to_value(u.kpc)
        
        # consider properties 
        particles['Metallicity'] = file['PartType4']['GFM_metallicity'][:].astype(np.float32)
        # include other properties as wish, be careful with the unit!!!
        
    # rotate to observing direction
    coords = rotate_coordinates(
        coordinates=particles['Coordinates'], 
        inclination=inclination, 
        azimuth=azimuth
    )
    
    # apply transformation defined in config
    # if rotate and shifts in config are 0, these lines can be removed
    coords = transform(coordinates=coords, 
                       rotate=transformation['rotate'], 
                       shiftPerpendicular=transformation['shiftPerpendicular'],
                       shiftParallel=transformation['shiftParallel'])
    
    # calculate the statistics for considered properties
    stats = calc_stats(
        coords=coords, 
        values=None, # None if for "count", otherwise values should be provided
        bins_perpendicular=bins_perpendicular, 
        bins_parallel=bins_parallel, 
        statistic='count') # statistic type, can be 'count', 'mean', 'std', 'sum'
    
    # stats = calc_stats(
    #     coords=coords,
    #     values=particles['Metallicity'],
    #     bins_perpendicular=bins_perpendicular,
    #     bins_parallel=bins_parallel,
    #     statistic='mean'
    # )
    
    unit = '1' # string for unit, if no unit, set to '1'
    
    return stats, unit

2. specify particle file, using the one saved in preprocess or manually input:  
    For manually input, call `input_particle_file` method, which accepts a particle file path as input, otherwise, the particle file saved in preprocess will be used.

In [ ]:
# calling this function will use the particle file specified
postprocess.input_particle_file(particle_file='your/particle/file/path')

3. Input the name of property and the corresponding functions:  
`get_truth_properties` method accepts `keep_defaults` (bool), `properties` (list), `functions` (list) as inputs.  

    - `keep_defaults`: if true keep the default properties, otherwise, only use the user-defined property functions
    - `properties`: list of property names
    - `functions`: list of property functions

In [ ]:
keep_defaults = False

properties = ['property_xx']
functions = [property_xx]

postprocess.get_truth_properties(
    keep_defaults=keep_defaults, 
    properties=properties, 
    functions=functions
)

Finally, the truth properties and their illustrations will be saved in `MSA_mock/Subhalo_{subhaloID}/`.